[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/elastic/geneve/main?labpath=docs/events_generation_walk-through.ipynb)

# Rule-based events generation walk-through

You can access an interactive version of this document by clicking on the "launch binder" badge above. You will be able to edit the `In [ ]` cells below by clicking in the grey area and executing them by pressing `Ctrl+Enter`.

## Preliminaries

The API for generating events is exposed by module `geneve.events_emitter`, the `SourceEvents` class provides the front-end for the most common use cases.

Here the module imports needed for the rest of this notebook.

In [1]:
import os; os.chdir('..')  # use the repo's root as base for importing local modules

from geneve.events_emitter import SourceEvents
from geneve.utils import load_schema, load_rules

from itertools import islice
from pathlib import Path

## Schema

Generating documents requires a specification of the fields types, such as `long` or `ip`. This is the duty of the _schema_.

An example of schema defining fields `x`, `y`, and `z` as float numbers is `{'x': {'type': 'float'}, 'y': {'type': 'float'}, 'z': {'type': 'float'}}`. Fields not defined in the schema are considered of type `keyword`, a kind of string.

The default schema is just `{}`, therefore every field is considered of type `keyword`. To change that, assign to `SourceEvents.schema` as follows:

In [2]:
SourceEvents.schema = {'x': {'type': 'float'}, 'y': {'type': 'float'}, 'z': {'type': 'float'}}

From now on every document generator will use the assigned schema.

In [3]:
a = SourceEvents()
a.schema

{'x': {'type': 'float'}, 'y': {'type': 'float'}, 'z': {'type': 'float'}}

In [4]:
b = SourceEvents()
b.schema

{'x': {'type': 'float'}, 'y': {'type': 'float'}, 'z': {'type': 'float'}}

It's also possible to specify the schema on a per-case basis.

In [5]:
c = SourceEvents(schema={'x': {'type': 'float'}})
c.schema

{'x': {'type': 'float'}}

The rest of the notebook assumes the rich and standardized [ECS](https://www.elastic.co/guide/en/ecs/current/ecs-reference.html) 8.1.0 as default schema, as loaded below.

In [6]:
SourceEvents.schema = load_schema('./etc/ecs-8.1.0.tar.gz', 'generated/ecs/ecs_flat.yml')

## Generate documents from queries

In the simplest form, documents can be generated as follows:

In [7]:
next(SourceEvents.from_query('process.name : *.exe'))

[Event(meta=None, doc={'process': {'name': 'rapziuzp.exe'}, '@timestamp': 1650461844954})]

Which is equivalent to

In [8]:
se = SourceEvents()
se.add_query('process.name : *.exe')
next(se)

[Event(meta=None, doc={'process': {'name': 'oyzjbqm.exe'}, '@timestamp': 1650461844960})]

In this second form, multiple queries can be added and the generated documents shall match either of them:

In [9]:
se = SourceEvents()
se.add_query('process.name : *.exe')
se.add_query('source.ip : 10.0.0.0/8')
next(se)

[Event(meta=None, doc={'source': {'ip': '10.166.138.200'}, '@timestamp': 1650461844969})]

The query language is guessed automatically. While the above are Kuery based, what follows uses EQL

In [10]:
se = SourceEvents()
se.add_query('process where process.name : "*.exe"')
next(se)

[Event(meta=None, doc={'process': {'name': 'b.exe'}, 'event': {'category': ['process']}, '@timestamp': 1650461844980})]

Currently only Kuery and EQL are supported though others will be added.

## Generate documents from rules

Similarily to the query cases, it's possible to generate documents from one or more rules.

In [11]:
# use the local copy of detection-rules 8.1.0
rules = load_rules('./etc/detection-rules-8.1.0.tar.gz', (
    'rules/network/command_and_control_port_26_activity.toml',
    'rules/network/command_and_control_telnet_port_activity.toml',
))

# load the rules
se = SourceEvents()
for rule in rules:
    se.add_rule(rule)

# generate one document
next(se)

[Event(meta=None, doc={'event': {'category': ['network']}, 'network': {'transport': 'tcp'}, 'destination': {'port': 23}, '@timestamp': 1650461845438})]

## Generate documents over and over

Adding queries and rules to a `SourceEvents` object triggers some operations like parsing the query, collecting the field constraints, trying to generate documents for the first time.

When it's important to avoid unnecessary computations, the result of such initial operations can be reused by preserving the `SourceEvents` object between the calls to `next`.

In [12]:
se = SourceEvents.from_query('process.name : (*.exe or *.dll)')

[next(se) for n in range(5)]

[[Event(meta=None, doc={'process': {'name': 'vficfhfumpjqfs.dll'}, '@timestamp': 1650461845444})],
 [Event(meta=None, doc={'process': {'name': 'givfjxueopsdrma.dll'}, '@timestamp': 1650461845444})],
 [Event(meta=None, doc={'process': {'name': 'wjfjtat.dll'}, '@timestamp': 1650461845444})],
 [Event(meta=None, doc={'process': {'name': 'u.exe'}, '@timestamp': 1650461845444})],
 [Event(meta=None, doc={'process': {'name': 'n.exe'}, '@timestamp': 1650461845444})]]

## Mappings of the generated documents

The `SourceEvents` object can build mappings describing all the fields used in the loaded queries and rules. For this task it's employed the same schema used for the documents generation.

In [13]:
# use the local copy of detection-rules 8.1.0
rules = load_rules('./etc/detection-rules-8.1.0.tar.gz', (
    'rules/network/command_and_control_port_26_activity.toml',
    'rules/network/command_and_control_telnet_port_activity.toml',
))

# load the rules
se = SourceEvents()
for rule in rules:
    se.add_rule(rule)

# generate the mappings
se.mappings()

{'properties': {'event': {'properties': {'dataset': {'type': 'keyword'},
    'category': {'type': 'keyword'}}},
  '@timestamp': {'type': 'date'},
  'network': {'properties': {'transport': {'type': 'keyword'}}},
  'destination': {'properties': {'port': {'type': 'long'}}}}}

## Query validation

In [14]:
try:
    SourceEvents.from_query('destination.port < 1024 and (destination.port > 512 or destination.port > 1024)')
except Exception as e:
    print(e)

Unsolvable constraints: destination.port (empty solution space, 1025 <= x <= 1023)


## Using as iterator

The `SourceEvents` class implements the iterator protocol, as the usage of `next` for generating new documents anticipated. Because documents can be generated indefinitely, the iterator is infinite and therefore some care is needed.

Uses like `list(se)` or `for docs in se: print(docs)` are troublesome, the first would sooner or later exhaust all the resources, the second would never terminate spontaneously. You then need to ask yourself how many documents you need or what other conditions will break the loop.

As example, this prints 10 documents:

In [15]:
se = SourceEvents.from_query('process.name : (*.exe or *.dll)')

for docs in islice(se, 10):
    print(docs)

[Event(meta=None, doc={'process': {'name': 'ntaxjajfnfxt.dll'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'nfmnobfqcndk.dll'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'n.exe'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'aeqdperf.dll'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'qqbgdwfresd.dll'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'gzyqbxe.exe'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'ew.exe'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'eifjkwvzhe.exe'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'sdr.dll'}, '@timestamp': 1650461845468})]
[Event(meta=None, doc={'process': {'name': 'ajvb.dll'}, '@timestamp': 1650461845469})]
